In [7]:
#Functions

import cv2
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
import pandas as pd
import numpy as np
import cv2
from matplotlib import pyplot as plt

# Lossless Encoder using Run-Length Encoding (RLE)
class LosslessEncoder:
    @staticmethod
    def encode(image):
        flat_image = image.flatten()
        bitstream = ""
        
        # Convert pixel values (0-255) into binary and concatenate
        for value in flat_image:
            bitstream += format(value, "08b")
        
        # Group into 8-bit chunks
        grouped_bitstream = [bitstream[i:i+8] for i in range(0, len(bitstream), 8)]
        return grouped_bitstream

# Lossless Decoder
class LosslessDecoder:
    @staticmethod
    def decode(bitstream, shape):
        # Flatten bitstream into a single binary string
        binary_string = "".join(bitstream)
        
        # Convert binary back into pixel values
        pixels = [int(binary_string[i:i+8], 2) for i in range(0, len(binary_string), 8)]
        return np.array(pixels, dtype=np.uint8).reshape(shape)

# Processing Two Images
def process_images(image1_path, image2_path):
    # Load the images
    image1 = cv2.imread(image1_path, cv2.IMREAD_GRAYSCALE)
    image2 = cv2.imread(image2_path, cv2.IMREAD_GRAYSCALE)
    
    # Step 1: Encode both images
    encoder = LosslessEncoder()
    bitstream1 = encoder.encode(image1)
    bitstream2 = encoder.encode(image2)
    
    # Step 2: Calculate bitstream lengths
    k1 = 8 * (len(bitstream1) - 1) + len(bitstream1[-1])  # Length of bitstream 1
    k2 = 8 * (len(bitstream2) - 1) + len(bitstream2[-1])  # Length of bitstream 2
    
    print(f"Bitstream Length for Image 1: k1 = {k1}")
    print(f"Bitstream Length for Image 2: k2 = {k2}")
    
    # Step 3: Decode both images
    decoder = LosslessDecoder()
    reconstructed_image1 = decoder.decode(bitstream1, image1.shape)
    reconstructed_image2 = decoder.decode(bitstream2, image2.shape)
    
    # Verify Lossless Compression
    assert np.array_equal(image1, reconstructed_image1), "Image 1: Reconstruction failed!"
    assert np.array_equal(image2, reconstructed_image2), "Image 2: Reconstruction failed!"
    
    # Step 4: Pad bitstreams to match flattened image size
    size1_diff = image1.size - len(bitstream1)
    size2_diff = image2.size - len(bitstream2)
    padded_bitstream1 = bitstream1 + ["I"] * size1_diff
    padded_bitstream2 = bitstream2 + ["I"] * size2_diff
    
    # Step 5: Prepare submission DataFrame
    submission = pd.DataFrame({
        "CompressedImage1": padded_bitstream1,
        "CompressedImage2": padded_bitstream2,
        "Image1": reconstructed_image1.flatten(),
        "Image2": reconstructed_image2.flatten(),
    })
    
    return submission, bitstream1, bitstream2, reconstructed_image1, reconstructed_image2

# Paths to the images (replace with your file paths)
image1_path = "path_to_image1.tiff"
image2_path = "path_to_image2.tiff"

# Process the images and create the submission DataFrame
submission, bitstream1, bitstream2, reconstructed_image1, reconstructed_image2 = process_images(image1_path, image2_path)

# Save the submission DataFrame as CSV (if needed)
submission.to_csv("submission.csv", index=False)

# Display a portion of the submission
print(submission.head())


In [9]:
# Paths to the images (replace with your file paths)

image1_path = "Watermarked_Image1.tiff"
image2_path = "Watermarked_Image2.tiff"

# Process the images and create the submission DataFrame
submission, bitstream1, bitstream2, reconstructed_image1, reconstructed_image2 = process_images(image1_path, image2_path)

# Save the submission DataFrame as CSV (if needed)
submission.to_csv("submission.csv", index=False)

# Display a portion of the submission
print(submission.head())


Compression Ratio for Image 1: 0.5785
Compression Ratio for Image 2: 0.8865
  CompressedImage1 CompressedImage2  Image1  Image2
0        (190, 16)       (255, 466)     190     255
1        (191, 14)         (254, 3)     190     255
2         (192, 2)         (255, 1)     190     255
3         (191, 2)         (254, 1)     190     255
4         (192, 1)         (253, 2)     190     255
